# KMeans

In [6]:
import cluster_data_pca
from cluster_data_pca import run_clustering, normalize_data, unnormalize
import numpy as np
import pandas as pd
import os
from kmeans import k_means
from cluster_plotter import ClusterPlotter
from clustering_utils_pca import ClusterData
import cluster_plotter
import high_dim_analysis
import seaborn as sns
import matplotlib.pyplot as plt
import my_dbcv_module
import scores 

standard_year_ranges = {
    f"{start}-{start + 3}": np.arange(start, start + 4)
    for start in [2002, 2006, 2010, 2014, 2019]
}
standard_year_ranges["2019-2023"] = np.arange(2019, 2024)
running_ranges = cluster_data_pca.generate_running_year_ranges(2002, 2023, 4)

binned_data = cluster_data_pca.bin_data_for_clustering(running_ranges, print_res=False)

plot_dir = "Images\\k_means_running_bins_pca"
os.makedirs(plot_dir, exist_ok=True)
cluster_plotter.clear_directory(plot_dir)

for cluster_data, year_range in binned_data:
    print(f"\nRunning K-Means for Year Range: {year_range}")

    data_array = np.array([cluster_data.ecc, cluster_data.mag_obj, cluster_data.sem_maj,
                            cluster_data.diameter, cluster_data.inc, cluster_data.raan]).T
    feature_names = ["Eccentricity e", "Object magnitude [mag]", "Semi major axis [km]",
                     "Diameter [m]", "Inclination [°]", "RAAN [°]"]

    normalized_data, data_min, data_max = normalize_data(data_array)

    k_values = [3, 4, 5, 6, 7]
    results = []

    for k in k_values:
        result_kmeans, time_kmeans, n_clusters_kmeans, points_per_cluster_kmeans, metrics_kmeans = \
            run_clustering(
                k_means, f"K-means (k={k})", normalized_data, data_min, data_max, k, init='kmeans++'
            )

        labels = result_kmeans.labels
        # Compute DBCV score using custom Rust module
        dbcv_score = scores.DBCV_score_rust(result_kmeans)

        results.append({
            "Year Range": year_range,
            "k": k,
            "Runtime (s)": f"{time_kmeans:.3f}",
            "Clusters": n_clusters_kmeans,
            "Points per Cluster": points_per_cluster_kmeans,
            "DBCV Score": f"{dbcv_score:.3f}"
        })

        # Plot diagnostics
        high_dim_analysis.plot_correlation_heatmap(
            cluster_data,
            output_folder=plot_dir,
            filename=f"corr_{year_range}_k{k}.png"
        )
        high_dim_analysis.plot_mutual_information(
            cluster_data,
            labels,
            output_folder=plot_dir,
            filename=f"mi_{year_range}_k{k}.png"
        )

        from sklearn.feature_selection import f_classif
        X = np.array([cluster_data.ecc, cluster_data.mag_obj, cluster_data.sem_maj,
                      cluster_data.diameter, cluster_data.inc, cluster_data.raan]).T
        valid = labels != -1
        if np.any(valid) and len(np.unique(labels[valid])) > 1:
            f_vals, _ = f_classif(X[valid], labels[valid])
            mean_f = np.mean(f_vals)
        else:
            mean_f = 0.0

        results[-1]["Mean ANOVA F"] = f"{mean_f:.3f}"

    df = pd.DataFrame(results)
    df["Mean ANOVA F"] = pd.to_numeric(df["Mean ANOVA F"], errors="coerce")
    df_sorted = df.sort_values(by="Mean ANOVA F", ascending=False)
    best_params = df_sorted.iloc[0]
    best_k = best_params["k"]

    best_result, _, _, _, _ = run_clustering(
        k_means, f"K-means (k={best_k})", normalized_data, data_min, data_max, int(best_k), init='kmeans++'
    )
    best_labels = best_result.labels

    df_plot = pd.DataFrame(data_array, columns=feature_names)
    df_plot['cluster'] = best_labels.astype(str)

    pairplot = sns.pairplot(df_plot, hue='cluster', diag_kind='kde', plot_kws={'alpha': 0.6, 's': 8})
    pairplot.fig.suptitle(f"K-Means Clusters for Year Range {year_range}", y=1.02)
    pairplot.savefig(os.path.join(plot_dir, f"pairplot_{year_range}_k{best_k}.png"))
    plt.close(pairplot.fig)

    high_dim_analysis.plot_anova_f_values(
        cluster_data,
        best_labels,
        output_folder=plot_dir,
        filename=f"anova_best_{year_range}_k{best_k}.png",
        title=f"ANOVA F-values (Best KMeans Parameters), k = {int(best_k)}"
    )
    print("Best KMeans parameters based on ANOVA F-value:")
    print(best_params)

    display(df)



Running K-Means for Year Range: 2002-2005
Runtime for k_means: 0.018001 seconds
Runtime for k_means: 0.029042 seconds
Runtime for k_means: 0.128654 seconds
Runtime for k_means: 0.151001 seconds
Runtime for k_means: 0.221861 seconds
Runtime for k_means: 0.029860 seconds


c:\Users\fionu\OneDrive\Dokumente\Daten Fiona\AIUB\Code\AIUB-Backup\clustering\high_dim_analysis.py:43: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=f_vals, y=features, palette="crest", legend = False)


Best KMeans parameters based on ANOVA F-value:
Year Range                              2002-2005
k                                               3
Runtime (s)                                 0.018
Clusters                                        3
Points per Cluster    {0: 3672, 1: 3149, 2: 1216}
DBCV Score                                 -0.832
Mean ANOVA F                            27762.699
Name: 0, dtype: object


,Year Range,k,Runtime (s),Clusters,Points per Cluster,DBCV Score,Mean ANOVA F
0,2002-2005,3,0.018,3,"{0: 3672, 1: 3149, 2: 1216}",-0.832,27762.699
1,2002-2005,4,0.029,4,"{0: 3155, 1: 1359, 2: 2315, 3: 1208}",-0.959,19750.659
2,2002-2005,5,0.129,5,"{0: 1669, 1: 1208, 2: 3154, 3: 772, 4: 1234}",-0.966,15255.199
3,2002-2005,6,0.151,6,"{0: 1236, 1: 1495, 2: 780, 3: 1658, 4: 2354, 5...",-0.967,12572.349
4,2002-2005,7,0.222,7,"{0: 786, 1: 4360, 2: 307, 3: 294, 4: 1102, 5: ...",-0.750,12223.131



Running K-Means for Year Range: 2003-2006
Runtime for k_means: 0.075371 seconds
Runtime for k_means: 0.065014 seconds
Runtime for k_means: 0.054575 seconds
Runtime for k_means: 0.090877 seconds
Runtime for k_means: 0.058519 seconds
Runtime for k_means: 0.011589 seconds


...

,Year Range,k,Runtime (s),Clusters,Points per Cluster,DBCV Score,Mean ANOVA F
0,2019-2022,3,0.031,3,"{0: 2286, 1: 352, 2: 971}",-0.788,5694.683
1,2019-2022,4,0.015,4,"{0: 1316, 1: 481, 2: 1303, 3: 509}",-0.883,3617.165
2,2019-2022,5,0.008,5,"{0: 1673, 1: 330, 2: 501, 3: 598, 4: 507}",-0.829,7332.618
3,2019-2022,6,0.023,6,"{0: 563, 1: 429, 2: 188, 3: 435, 4: 1707, 5: 287}",-0.934,4845.388
4,2019-2022,7,0.047,7,"{0: 304, 1: 965, 2: 257, 3: 498, 4: 759, 5: 25...",-0.927,2434.801



Running K-Means for Year Range: 2020-2023
Runtime for k_means: 0.010997 seconds
Runtime for k_means: 0.021108 seconds
Runtime for k_means: 0.025953 seconds
Runtime for k_means: 0.032569 seconds
Runtime for k_means: 0.018064 seconds
Runtime for k_means: 0.007052 seconds


c:\Users\fionu\OneDrive\Dokumente\Daten Fiona\AIUB\Code\AIUB-Backup\clustering\high_dim_analysis.py:43: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=f_vals, y=features, palette="crest", legend = False)


Best KMeans parameters based on ANOVA F-value:
Year Range                                    2020-2023
k                                                     4
Runtime (s)                                       0.021
Clusters                                              4
Points per Cluster    {0: 2775, 1: 411, 2: 601, 3: 585}
DBCV Score                                       -0.717
Mean ANOVA F                                  11691.106
Name: 1, dtype: object


,Year Range,k,Runtime (s),Clusters,Points per Cluster,DBCV Score,Mean ANOVA F
0,2020-2023,3,0.011,3,"{0: 1079, 1: 505, 2: 2788}",-0.852,7239.932
1,2020-2023,4,0.021,4,"{0: 2775, 1: 411, 2: 601, 3: 585}",-0.717,11691.106
2,2020-2023,5,0.026,5,"{0: 900, 1: 1572, 2: 531, 3: 747, 4: 622}",-0.900,3498.851
3,2020-2023,6,0.033,6,"{0: 528, 1: 497, 2: 1077, 3: 622, 4: 900, 5: 748}",-0.931,3492.068
4,2020-2023,7,0.018,7,"{0: 410, 1: 539, 2: 183, 3: 560, 4: 576, 5: 15...",-0.873,6299.979


# KMeans
The features used by the clustering are determined by applying pca on the raw dataset.   
After the clustering, we reapply the PCA and plot the clusters in the PC1 / PC2 plane and make the pairplot with the clustered data. 

In [8]:
import cluster_data_pca
from cluster_data_pca import run_clustering, normalize_data, unnormalize
import numpy as np
import pandas as pd
import os
from kmeans import k_means
from clustering_utils_pca import ClusterData
import cluster_plotter
import high_dim_analysis
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import scores

standard_year_ranges = {
    f"{start}-{start + 3}": np.arange(start, start + 4)
    for start in [2002, 2006, 2010, 2014, 2019]
}
standard_year_ranges["2019-2023"] = np.arange(2019, 2024)
running_ranges = cluster_data_pca.generate_running_year_ranges(2002, 2023, 4)

data_set = {"2005-2008": np.arange(2005, 2009)}
binned_data = cluster_data_pca.bin_data_for_clustering(data_set, print_res=False)
plot_dir = r"Images\k_means_running_bins_pcabased_pcaevaluated"
os.makedirs(plot_dir, exist_ok=True)
cluster_plotter.clear_directory(plot_dir)

for cluster_data, year_range in binned_data:
    print(f"\nRunning K-Means for Year Range: {year_range}")

    X = np.vstack([cluster_data.ecc, cluster_data.mag_obj,
                   cluster_data.sem_maj, cluster_data.diameter,
                   cluster_data.inc, cluster_data.raan]).T
    feature_names = ["Eccentricity e", "Object magnitude [mag]",
                     "Semi major axis [km]", "Diameter [m]",
                     "Inclination [°]", "RAAN [°]"]

    normalized_data, data_min, data_max = normalize_data(X)
    k_values = [3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
    results = []

    for k in k_values:
        result, t, n_clusters, pts_per_cluster, _ = run_clustering(
            k_means, f"K-means (k={k})",
            normalized_data, data_min, data_max, k, init='kmeans++'
        )
        labels = result.labels
        #dbcv = scores.DBCV_score_rust(result)

        results.append({
            "Year Range": year_range,
            "k": k,
            "Runtime (s)": f"{t:.3f}",
            "Clusters": n_clusters,
            "Points per Cluster": pts_per_cluster
        })
    
        df = pd.DataFrame(results)
        # Select best by DBCV
        #best = df.loc[df['DBCV Score'].astype(float).idxmax()]
        #best_k = int(best['k'])

        # Pairplot
        dfp = pd.DataFrame(X, columns=feature_names)
        dfp['cluster'] = labels.astype(str)
        pp = sns.pairplot(dfp, hue='cluster', diag_kind='kde', plot_kws={'alpha':0.6,'s':8})
        pp.fig.suptitle(f"K-Means PCA Clusters {year_range}", y=1.02)
        pp.savefig(os.path.join(plot_dir, f"pairplot_{year_range}_k{k}.png"))
        plt.close(pp.fig)

    """# PCA scatter
    scaler = StandardScaler()
    comps = PCA(2).fit_transform(scaler.fit_transform(X))
    plt.figure(figsize=(8,6))
    plt.scatter(comps[:,0], comps[:,1], c=labels, s=8, alpha=0.7)
    plt.xlabel('PC1'); plt.ylabel('PC2')
    plt.title(f'PCA PC1 vs PC2 {year_range}')
    plt.tight_layout()
    plt.savefig(os.path.join(plot_dir, f"pca_pc1_pc2_{year_range}_k{best_k}.png"))
    plt.close()"""


Running K-Means for Year Range: 2005-2008
Runtime for k_means: 0.025048 seconds
Runtime for k_means: 0.035145 seconds
Runtime for k_means: 0.040534 seconds
Runtime for k_means: 0.036052 seconds
Runtime for k_means: 0.105766 seconds
Runtime for k_means: 0.140035 seconds
Runtime for k_means: 0.122832 seconds
Runtime for k_means: 0.178582 seconds
Runtime for k_means: 0.145988 seconds
Runtime for k_means: 0.311936 seconds


# DBSCAN

In [8]:
import cluster_data_pca
from cluster_data_pca import run_clustering, normalize_data, unnormalize
import numpy as np
import pandas as pd
import os
from DBSCAN import dbscan_clustering
import cluster_plotter
import high_dim_analysis
import seaborn as sns
import matplotlib.pyplot as plt
import scores

running_ranges = cluster_data_pca.generate_running_year_ranges(2002, 2023, 4)
binned = cluster_data_pca.bin_data_for_clustering(running_ranges, print_res=False)
plot_dir = r"Images\dbscan_tests_running_bins_pca"
os.makedirs(plot_dir, exist_ok=True)
cluster_plotter.clear_directory(plot_dir)

for data_obj, yr in binned:
    print(f"\nRunning DBSCAN for Year Range: {yr}")
    X = np.vstack([data_obj.ecc, data_obj.mag_obj,
                   data_obj.sem_maj, data_obj.diameter,
                   data_obj.inc, data_obj.raan]).T
    normalized, mn, mx = normalize_data(X)

    eps_vals = [0.02, 0.01, 0.015]
    min_samps = [10,15,25,30]
    res_list = []

    for eps in eps_vals:
        for ms in min_samps:
            result, t, n_cl, pts_cl, _ = run_clustering(
                dbscan_clustering, "DBSCAN", normalized, mn, mx,
                eps=eps, min_samples=ms
            )
            labels = result.labels
            dbcv = scores.DBCV_score_rust(result)
            noise = np.sum(labels == -1)
            res_list.append({
                "Year Range": yr,
                "eps": eps,
                "min_samples": ms,
                "Runtime (s)": f"{t:.3f}",
                "Clusters": n_cl,
                "Points per Cluster": pts_cl,
                "Noise Points": noise,
                "DBCV Score": f"{dbcv:.3f}"
            })
            high_dim_analysis.plot_correlation_heatmap(
                data_obj, plot_dir, f"corr_{yr}_eps{eps}_ms{ms}.png"
            )
    
    df = pd.DataFrame(res_list)
    best = df.loc[df['DBCV Score'].astype(float).idxmax()]
    beps, bms = best['eps'], best['min_samples']

    final, _, _, _, _ = run_clustering(
        dbscan_clustering, "DBSCAN", normalized, mn, mx,
        eps=beps, min_samples=bms
    )
    labels = final.labels

    # Pairplot
    dfp = pd.DataFrame(X, columns=["e","mag","a","d","i","r"])
    dfp['cluster'] = labels.astype(str)
    pp = sns.pairplot(dfp, hue='cluster', diag_kind='kde', plot_kws={'alpha':0.6,'s':8})
    pp.fig.suptitle(f"DBSCAN Clusters {yr}", y=1.02)
    pp.savefig(os.path.join(plot_dir, f"pairplot_{yr}_eps{beps}_ms{bms}.png"))
    plt.close(pp.fig)


Running DBSCAN for Year Range: 2002-2005
Runtime for dbscan_clustering: 0.187283 seconds
Runtime for dbscan_clustering: 0.223996 seconds
Runtime for dbscan_clustering: 0.205609 seconds
Runtime for dbscan_clustering: 0.165021 seconds
Runtime for dbscan_clustering: 0.121889 seconds
Runtime for dbscan_clustering: 0.180750 seconds
Runtime for dbscan_clustering: 0.128001 seconds
Runtime for dbscan_clustering: 0.054032 seconds
Runtime for dbscan_clustering: 0.165889 seconds
Runtime for dbscan_clustering: 0.142087 seconds
Runtime for dbscan_clustering: 0.145278 seconds
Runtime for dbscan_clustering: 0.144871 seconds
Runtime for dbscan_clustering: 0.120252 seconds

Running DBSCAN for Year Range: 2003-2006
Runtime for dbscan_clustering: 0.217514 seconds
Runtime for dbscan_clustering: 0.213635 seconds
Runtime for dbscan_clustering: 0.214136 seconds
Runtime for dbscan_clustering: 0.244580 seconds
Runtime for dbscan_clustering: 0.063986 seconds
Runtime for dbscan_clustering: 0.162415 seconds
Runt

# DBSCAN
The features used by the clustering are determined by applying pca on the raw dataset.   
After the clustering, we reapply the PCA and plot the clusters in the PC1 / PC2 plane and make the pairplot with the clustered data. 

In [1]:
import cluster_data_pca
from cluster_data_pca import run_clustering, normalize_data, unnormalize
import numpy as np
import pandas as pd
import os
from DBSCAN import dbscan_clustering
import cluster_plotter
import high_dim_analysis
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import scores

running_ranges = cluster_data_pca.generate_running_year_ranges(2002, 2023, 4)
bins = cluster_data_pca.bin_data_for_clustering(running_ranges, print_res=False)
plot_dir = r"Images\\dbscan_tests_running_bins_pcabased_pcaevaluated"
os.makedirs(plot_dir, exist_ok=True)
cluster_plotter.clear_directory(plot_dir)

for data_obj, yr in bins:
    print(f"\nRunning DBSCAN for Year Range: {yr}")
    X = np.vstack([data_obj.ecc, data_obj.mag_obj,
                   data_obj.sem_maj, data_obj.diameter,
                   data_obj.inc, data_obj.raan]).T
    norm, mn, mx = normalize_data(X)

    eps_vals = [0.02, 0.01, 0.015]
    ms_vals = [10,15,25,30]
    res = []

    for eps in eps_vals:
        for ms in ms_vals:
            result, t, ncl, pts, _ = run_clustering(
                dbscan_clustering, "DBSCAN", norm, mn, mx,
                eps=eps, min_samples=ms
            )
            lbl = result.labels
            dbcv = scores.DBCV_score_rust(result)
            res.append({
                "Year Range": yr,
                "eps": eps,
                "min_samples": ms,
                "Runtime (s)": f"{t:.3f}",
                "Clusters": ncl,
                "Points per Cluster": pts,
                "DBCV Score": f"{dbcv:.3f}"
            })
    
    df = pd.DataFrame(res)
    best = df.loc[df['DBCV Score'].astype(float).idxmax()]
    beps, bms = best['eps'], best['min_samples']

    final, _, _, _, _ = run_clustering(
        dbscan_clustering, "DBSCAN", norm, mn, mx,
        eps=beps, min_samples=bms
    )
    labels = final.labels

    # Pairplot & PCA scatter
    dfp = pd.DataFrame(X, columns=["e","mag","a","d","i","r"])
    dfp['cluster'] = labels.astype(str)
    pp = sns.pairplot(dfp, hue='cluster', diag_kind='kde', plot_kws={'alpha':0.6,'s':8})
    pp.fig.suptitle(f"DBSCAN PCA Clusters {yr}", y=1.02)
    pp.savefig(os.path.join(plot_dir, f"pairplot_{yr}_eps{beps}_ms{bms}.png"))
    plt.close(pp.fig)

    comps = PCA(2).fit_transform(StandardScaler().fit_transform(X))
    plt.figure(figsize=(8,6))
    plt.scatter(comps[:,0], comps[:,1], c=labels, s=10, alpha=0.7)
    plt.xlabel('PC1'); plt.ylabel('PC2')
    plt.title(f'PCA DBSCAN Clusters {yr}')
    plt.tight_layout()
    plt.savefig(os.path.join(plot_dir, f"pca_clusters_{yr}_eps{beps}_ms{bms}.png"))
    plt.close()



Running DBSCAN for Year Range: 2002-2005
Runtime for dbscan_clustering: 0.075176 seconds
Runtime for dbscan_clustering: 0.083376 seconds
Runtime for dbscan_clustering: 0.083105 seconds
Runtime for dbscan_clustering: 0.066313 seconds
Runtime for dbscan_clustering: 0.050082 seconds
Runtime for dbscan_clustering: 0.066332 seconds
Runtime for dbscan_clustering: 0.050448 seconds
Runtime for dbscan_clustering: 0.049724 seconds
Runtime for dbscan_clustering: 0.066506 seconds
Runtime for dbscan_clustering: 0.046329 seconds
Runtime for dbscan_clustering: 0.050279 seconds
Runtime for dbscan_clustering: 0.065580 seconds
Runtime for dbscan_clustering: 0.066485 seconds

Running DBSCAN for Year Range: 2003-2006
Runtime for dbscan_clustering: 0.103164 seconds
Runtime for dbscan_clustering: 0.080681 seconds
Runtime for dbscan_clustering: 0.084422 seconds
Runtime for dbscan_clustering: 0.099679 seconds
Runtime for dbscan_clustering: 0.051666 seconds
Runtime for dbscan_clustering: 0.071409 seconds
Runt